## Introduction

This chapter addresses three major topics that are closely related:
- Python's `async def`, `await`, `async with`, and `async for` constructs
- Objects supporting those constructs: native coroutines and asynchronous variants of context managers, iterables, generators, and comprehensions
- `asyncio` and other asynchronous libraries

## A Few Definitions
- <span style="color:skyblue">***Native coroutine***</span>: A coroutine function defined with `async def`. You can delegate from a native coroutine to another native coroutine using the `await` keyword, similar to how classic coroutines use `yield` from. The `async def` statement always defines a native coroutine, even if the await keyword is not used in its body. The `await` keyword cannot be used outside of a native coroutine
- <span style="color:skyblue">***Classic coroutine***</span>: A generator function that consumes data sent to it via `my_coro.send(data)` calls, and reads that data by using `yield` in an expression. Classic coroutines can delegate to other classic coroutines using `yield` from. Classic coroutines cannot be driven by `await`, and are no longer supported by `asyncio`.
- <span style="color:skyblue">***Generator-based coroutine***</span>: A generator function decorated with `@types.coroutine` — introduced in Python 3.5. That decorator makes the generator compatible with the new `await` keyword.
- <span style="color:skyblue">***Asynchronous generator***</span>: A generator function defined with `async def` and using `yield` in its body. It returns an asynchronous generator object that provides `__anext__`, a coroutine method to retrieve the next item.

## An `asyncio` Example: Probing Domains

We want to write a script (`blogdom.py`) to concurrently check if domains `<name>.dev` are available for our Python blog. The output should be  
```bash
python3 blogdom.py
with.dev
+ elif.dev
+ def.dev
from.dev
```
Compared to checking domains sequentially, probing DNS via native coroutine objects should be much faster, almost the same as the time to check the single slowest DNS, instead of the sum of all responses

In [1]:
import asyncio
import socket
from keyword import kwlist
import nest_asyncio

nest_asyncio.apply()  # allow nested usage of asyncio. Only needs for jupyter notebook

MAX_KEYWORD_LEN = 4  # Max length of keyword for domains, because shorter is better.


async def probe(domain: str) -> tuple[str, bool]:
    """
    Returns a tuple with the domain name and a boolean; True means the domain resolved
    Returning the domain name will make it easier to display the results
    """
    loop = asyncio.get_running_loop()  # Get a reference to the asyncio event loop, so we can use it next.
    try:
        await loop.getaddrinfo(domain, None)  # loop.getaddrinfo coroutine returns a five-part tuple of
                                              # parameters to connect to the given address using a socket
                                              # In this example, we don’t need the result. 
                                              # If we got it, the domain resolves; otherwise, it doesn’t.
    except socket.gaierror:  # `gai` stands for get address info 
        return (domain, False)
    return (domain, True)


async def main() -> None:  # main must be a coroutine, so that we can use await in it.
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)  # Generator to yield Python 
                                                                 # keywords with length up to `MAX_KEYWORD_LEN`
    domains = (f'{name}.dev'.lower() for name in names)  # Generator to yield domain names with the `.dev` suffix
    coros = [probe(domain) for domain in domains]  # Build a list of coroutine objects
    for coro in asyncio.as_completed(coros):  # `asyncio.as_completed` is a generator that yields 
                                              # coroutines that return the results of the 
                                              # coroutines passed to it in the order they are completed
        domain, found = await coro  # At this point, we know the coroutine is done 
                                    # because that’s how as_completed works
        mark = '+' if found else ' '
        print(f'{mark} {domain}')


asyncio.run(main())  # starts the event loop and returns only when the event loop exits.

+ as.dev
+ in.dev
+ try.dev
+ def.dev
+ from.dev
+ and.dev
  is.dev
  true.dev
  with.dev
  elif.dev
  pass.dev
  for.dev
  if.dev
+ not.dev
  none.dev
  or.dev
  else.dev
+ del.dev


**Low-level Explainations:**

<span style="color:lightgreen">***Using the syntax `await loop.getaddrinfo(...)` avoids blocking because `await` suspends the current coroutine object. For example, during the execution of the `probe('if.dev')` coroutine, a new coroutine object is created by `getaddrinfo('if.dev', None)`. Awaiting it starts the low-level `addrinfo` query and yields control back to the event loop, not to the `probe(‘if.dev’)` coroutine, which is suspended. The event loop can then drive other pending coroutine objects, such as `probe('or.dev')`. When the event loop gets a response for the `getaddrinfo('if.dev', None)` query, that specific coroutine object resumes and returns control back to the `probe('if.dev')` — which was suspended at `await` — and can now handle a possible exception and return the result tuple.***</span>

## Guido's Trick to Read Async Code

<span style="color:skyblue">***The trick is to pretend that the `async` and `await` keywords are not there. If you do that, you’ll realize that coroutines read like plain old sequential functions, but they just magically never block.***</span>

## Awaitables

<span style="color:skyblue">*The `for` keyword works with iterables. The `await` keyword works with awaitables.*</span> As an end user of `asyncio`, these are the awaitables you will see on a daily basis:  
- <span style="color:skyblue">*A native coroutine object, which you get by calling a native coroutine function. We use `await other_coro()` to run `other_coro` right now and `wait` for its completion because we need its result before we can proceed.*</span>
- <span style="color:skyblue">*An `asyncio.Task`, which you usually get by passing a coroutine object to `asyncio.create_task()`. However, end-user code does not always need to `await` on a `Task`. We use `asyncio.create_task(one_coro())` to schedule `one_coro` for concurrent execution, without waiting for its return.*</span>

Awaitables are 
- Objects with an `__await__` method that returns an iterator, e.g. an `asyncio.Future` instance
- Objects written in other languages using the Python / C API with a `tp_as_async.am_await` function, returning an iterator

## Downloading with `asyncio` and `HTTPX`

Look into the code of `flags_asyncio`:

```python
import asyncio

from httpx import AsyncClient

from flags import BASE_URL, save_flag, main


async def download_one(client: AsyncClient, cc: str):
    """
    `download_one` must be a native coroutine, so it can await on `get_flag` — 
    which does the HTTP request. Then it displays the code of the downloaded flag, and
    saves the image.
    """
    image = await get_flag(client, cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc


async def get_flag(client: AsyncClient, cc: str) -> bytes:
    """
    get_flag needs to receive the AsyncClient to make the request.
    """
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=6.1,
                follow_redirects=True)  # The get method of an httpx.AsyncClient 
                                        # instance returns a ClientResponse
                                        # object that is also an asynchronous context manager
    return resp.read()  # Network I/O operations are implemented as coroutine methods, so they are
                        # driven asynchronously by the asyncio event loop
```

The code delegates to the `httpx` coroutines explicitly through `await` or implicitly through the special methods of the asynchronous context managers, such as `AsyncClient` and `ClientResponse`

## The Secret of Native Coroutines: Humble Generators

Using `httpx`, we don't need to use `.send()` calls or `yield` expressions. The code sits beteen the `asyncio` library and the async libraries we are using, e.g. `httpx`

<img src="../images/async.png" style="width: 50%;">.  

Under the hood, the asyncio event loop makes the `.send` calls that drive your coroutines, and your coroutines `await` on other coroutines, including library coroutines. Using functions like `asyncio.gather` and `asyncio.create_task`, you can start multiple concurrent await channels, enabling concurrent execution of multiple I/O operations driven by a single event loop, in a single thread.

## The All-or-Nothing Problem

<span style="color:skyblue">***For peak performance with `asyncio`, we must replace every function that does I/O with an asynchronous version that is activated with `await` or `asyncio.create_task`, so that control is given back to the event loop while the function waits for I/O. If you can’t rewrite a blocking function as a coroutine, you should run it in a separate thread or process.***</span>

<span style="color:orange">***“You rewrite all your code so none of it blocks or you’re just wasting your time.”***</span>. This is why in `flags_asyncio.py`, all functions (except for `download_many` which calls `asyncio.run`) are coroutines

## Asynchronous Context Managers

Using a `with` block, an object can be used to run code before and after the body if its class provides the `__enter__` and `__exit__` methods. Using `async with` statement, we can work with asynchronous context managers: objects implementing the `__aenter__` and `__aexit__` methods as coroutines. For example, A database transaction is a natural fit for the context manager protocol: the transaction has to be started, data is changed with connection.execute, and then a rollback or commit must happen, depending on the outcome of the changes:

```python
async with connection.transaction():
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")
```

## Enhancing the asyncio Downloader


### Using `asyncio.as_completed` and a `Thread`
Below, we run an attempt to get 100 flags (`-al 100`) from the ERROR server, using 100 concurrent requests (`-m 100`). The errors in the result are either HTTP 418 or time-out errors — the expected (mis)behavior of the `slow_server.py`

```bash
cd flags_asyncio
python slow_server.py 8004 --error-rate .25  # set up the ERROR local server
python3 flags2_asyncio.py -s ERROR -al 100 -m 100
```

Note: <span style="color:skyblue">*All network I/O is done with coroutines in `asyncio`, but not file I/O. However, file I/O is also "blocking" — in the sense that reading/writing files takes thousands of times longer than reading/writing to RAM. If you’re using Network-Attached Storage (NAS), it may even involve network I/O under the covers. Since Python 3.9, the `asyncio.to_thread` coroutine makes it easy to delegate file I/O to a thread pool provided by asyncio.*</span>

### Throttling Requests with a Semaphore

A semaphore is a synchronization primitive, more flexible than a lock. A semaphore can be held by multiple coroutines, with a configurable maximum number. This makes it ideal to throttle the number of active concurrent coroutines.  

There are three `Semaphore` classes in Python’s standard library: one in `threading`, another in `multiprocessing`, and a third one in `asyncio`. An `asyncio.Semaphore` has an internal counter that is decremented whenever we `await` on the `.acquire()` coroutine method, and incremented when we call the `.release()` method — which is not a coroutine because it never blocks. Awaiting on `.acquire()` causes no delay when the counter is greater than zero, but if the counter is zero, `.acquire()` suspends the awaiting coroutine until some other coroutine calls `.release()` on the same `Semaphore`, thus incrementing the counter. Instead of using those methods directly, it’s safer to use the semaphore as an asynchronous context manager:
```python
async with semaphore:
    image = await get_flag(client, base_url, cc)
```
The `Semaphore.__aenter__` coroutine method awaits for `.acquire()`, and its `__aexit__` coroutine method calls `.release()`.

### Making Multiple Requests for Each Download

Suppose you want to save each country flag with the name of the country and the country code, instead of just the country code. Now you need to make two HTTP requests per flag: one to get the flag image itself, the other to get the `metadata.json` file in the same directory as the image — that’s where the name of the country is recorded.

Coordinating multiple requests in the same task is easy in the threaded script: just make one request then the other, blocking the thread twice, and keeping both pieces of data (country code and name) in local variables, ready to use when saving the files. If you needed to do the same in an asynchronous script with callbacks, you needed nested functions so that the country code and name were available in their closures until you could save the file, because each callback runs in a different local scope. The `await` keyword provides relief from that, allowing you to drive the asynchronous requests one after the other, sharing the local scope of the driving coroutine.

Look into the `flags_asyncio/flags3_asyncio.py`

<span style="color:orange">*One challenge is to know when you have to use `await` and when you can’t use it. The answer in principle is easy: you `await` coroutines and other awaitables, such as `asyncio.Task` instances. But some APIs are tricky, mixing coroutines and plain functions in seemingly arbitrary ways.*</span>

## Delegating Tasks to Executors

NodeJS provides async APIs for all I/O, but not for network I/O. In Python, if you’re not careful, file I/O can seriously degrade the performance of asynchronous applications, because reading and writing to storage in the main thread blocks the event loop. We used `await asyncio.to_thread(save_flag, image, f'{cc}.gif')` to save the downloaded image to disk, which uses `asyncio.get_running_loop().run_in_executor()` under the hood. The main reason to pass an explict `Executor` to `loop.run_in_executor` is to employ a `ProcessPoolExecutor` if the function to execute is CPU intensive, so that it runs in a different Python process, avoiding contention for the GIL.

## Writing `asyncio` Servers

We’ll build a server-side Unicode character search utilities, first using HTTP with `FastAPI`, then using plain TCP with `asyncio` only. These servers let users query for Unicode characters based on words in their standard names from the unicodedata module

### Meet the Inverted Index for Unicode Character Search

An inverted index usually maps words to documents in which they occur. In the mojifinder examples, each “document” is one Unicode character. The `charindex.InvertedIndex` class indexes each word that appears in each character name in the Unicode database, and creates an inverted index stored in a `defaultdict`.

In [8]:
from mojifinder.charindex import InvertedIndex

idx = InvertedIndex()

print(idx.entries['CAT'])
print(idx.entries['FACE'])
print(idx.search('cat face'))

{'챁', '😺', '😽', '🐈', '😼', '😾', 'ꊶ', '𐇬', '😹', '😸', '🙀', '😿', '😻', '🐱'}
{'〠', '😺', '🤨', '😘', '🤭', '🌬', '🐯', '龜', '🫢', '🐮', '😚', '😟', '🤑', '🙃', '🫤', '🙅', '😾', '😍', '😤', '😽', '🫠', '🐼', '🥺', '🕐', '😢', '😛', '🌝', '😄', '🕠', '😶', '🥲', '🕣', '😈', '😦', '⚀', '🐺', '⚅', '😱', '🥰', '😞', '😧', '🥴', '🤬', '🌚', '🤥', '😩', '🥸', '🐸', '😀', '🤤', '😒', '🫣', '🐶', '🐵', '☹', '😰', '🐷', '😋', '😭', '🥱', '🤫', '🫥', '😴', '🐹', '🥵', '🙂', '😮', '😎', '🕧', '⚁', '😥', '🤗', '🕦', '𝨉', '😯', '😪', '🤐', '😨', '🙁', '🐴', '🙄', '🤪', '🐻', '😲', '⚄', '🤔', '🕡', '😃', '⚂', '😣', '🕙', '😊', '🤡', '🦒', '😳', '🤩', '🤮', '😂', '😕', '🤯', '😸', '🐱', '🦓', '🕝', '😝', '😹', '🕘', '🤒', '🤢', '🙆', '😌', '😐', '😉', '😷', '🤓', '🕥', '😜', '🕢', '🕒', '😙', '😆', '🤕', '☺', '⚃', '🕛', '😁', '😅', '🥳', '🐲', '🤖', '🕞', '🧐', '𝨇', '🦁', '🕜', '🦊', '🕑', '🙀', '🦄', '😿', '🕓', '⾯', '😖', '🥹', '🐭', '😡', '😑', '💆', '🫡', '☻', '😗', '🤦', '🕟', '😏', '😻', '🌜', '🕤', '🕗', '😼', '🤧', '𝨈', '🕖', '😇', '🙎', '🤠', '😔', '😵', '🥶', '😓', '🕚', '🌛', '😬', '😠', '🌞', '🕔', '🕕', '🐰', '😫'}
{'😺', '😽', '😼', '😹', '😸', '🙀', '😻', '😿

### A FastAPI Webservice

The FastAPI webserver was written in `mojifinder/web_mojifinder.py`. Demo run with 
```bash
cd mojifinder
uvicorn web_mojifinder:app --reload
```

The frontend is a very simple single page application (SPA): after the initial HTML download, the UI is updated by client-side JavaScript communicating with the server, which looks like below 

<img src="../images/fastapi-mojifinder-server.png" style="width: 60%;">.  

FastAPI is designed to implement backends for SPA and mobile apps, which mostly consist of *web API end points returning JSON responses instead of server-rendered HTML*.

In our server, there are only 2 endpoints: 
1. The root URL `/` which sends the `form.html` file with 81 lines of html and js code to communicate with the server and fill a table with the results
2. The `/search` URL which returns a generator of dictionary containing the characters and names <img src="../images/fastapi-cat-search.png" style="width: 60%;">.  

We can see that the  this `mojifinder/web_mojifinder.py` module has no direct calls to `asyncio`. However, FastAPI is built on the Starlette ASGI toolkit, which in turn uses `asyncio`. In a real app, most endpoints will query databases or hit other remote servers, so it is a critical advantage of FastAPI—and ASGI frameworks in general—to support coroutines that can take advantage of asynchronous libraries for network I/O.

### An `asyncio` TCP Server

Now, let's look at the `mojifinder/tcp_mojifinder.py` program which uses plain TCP to communicate with a client like Telnet or Netcat, so we could write it using `asyncio` without external dependencies — and without reinventing HTTP. To run the server, do

```bash
cd mojifinder
python tcp_mojifinder.py
```

Then, run a client with `telnet` and query, e.g. `cat`. The result is like below
<img src="../images/tcp-cat-search.png" style="width: 100%;">

## Asynchronous Iteration and Asynchronous Iterables

## async Beyond `asyncio`: `Curio`

## Type Hinting Asynchronous Objects

## How Async Works and How It Doesn’t

### Running Circles Around Blocking Calls

From a perspective, we can see blocking functions as ones that do file or network I/O, and argues that we can’t treat them as we treat nonblocking functions.

<img src="../images/data_reading_latency.png" style="width: 50%;">.

Since reading from disk or network is so long, the above table explains why a disciplined approach to asynchronous programming can lead to high-performance servers. The challenge is achieving that discipline. The first step is to recognize that “I/O bound system” is a fantasy.

### The Myth of I/O-Bound Systems

<span style="color:lightgreen">*A commonly repeated meme is that asynchronous programming is good for “I/O bound systems.” There are no “I/O-bound systems.”: You may have I/O-bound functions. Perhaps the vast majority of the functions in your system are I/O bound; i.e., they spend more time waiting for I/O than crunching data. While waiting, they cede control to the event loop, which can then drive some other pending task. But inevitably, any nontrivial system will have some parts that are CPU bound. Even trivial systems reveal that, under stress. Given that any nontrivial system will have CPU-bound functions, dealing with them is the key to success in asynchronous programming.*</span>

### Avoiding CPU-Bound Traps

If you’re using Python at scale, you should have some <span style="color:lightgreen">*automated tests designed specifically to detect performance regressions as soon as they appear*</span>. This is critically important with asynchronous code, but also relevant to threaded Python code — because of the GIL. If you wait until the slowdown starts bothering the development team, it’s too late. The fix will probably require some major makeover.

Here are some options for when you identify a CPU-hogging bottleneck:  
- <span style="color:skyblue">*Delegate the task to a Python process pool.*</span>
- <span style="color:skyblue">*Delegate the task to an external task queue. The external task queue should be chosen and integrated as soon as possible at the start of the project, so that nobody in the team hesitates to use it when needed*</span>
- <span style="color:skyblue">*Rewrite the relevant code in Cython, C, Rust, or some other language that compiles to machine code and interfaces with the Python/C API, preferably releasing the GIL.*</span>
- <span style="color:skyblue">*Decide that you can afford the performance hit and do nothing—but record the decision to make it easier to revert to it later.*</span>